In [4]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 6, Finished, Available, Finished)

In [14]:
Silver_Layer_path = "Files/Silver/Final_Sales_Data"
Gold_Layer_path = "Files/Gold"

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 16, Finished, Available, Finished)

In [2]:
merged_data_df = spark.read.parquet(Silver_Layer_path)
display(merged_data_df)

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2c4dcc9f-cc0e-49e6-9f00-af6506201bd3)

In [3]:
merged_data_df.count()

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 5, Finished, Available, Finished)

110000

In [7]:
# Creating Dim_Product
dim_product_df = merged_data_df.select("PRODUCT_NAME", "CATEGORY", "SUB_CATEGORY") \
    .distinct() \
    .withColumnRenamed("PRODUCT_NAME", "PRODUCT_NAME") \
    .withColumnRenamed("CATEGORY", "CATEGORY") \
    .withColumnRenamed("SUB_CATEGORY", "SUB_CATEGORY")

# Creating Dim_Customer
dim_customer_df = merged_data_df.select("CUSTOMER_NAME", "REGION", "CUSTOMER_TYPE") \
    .distinct() \
    .withColumnRenamed("CUSTOMER_NAME", "CUSTOMER_NAME") \
    .withColumnRenamed("REGION", "REGION") \
    .withColumnRenamed("CUSTOMER_TYPE", "CUSTOMER_TYPE")

# Create Dim_Date
dim_date_df = merged_data_df.select("SALES_DATE").distinct() \
    .withColumn("YEAR", year("SALES_DATE")) \
    .withColumn("MONTH", month("SALES_DATE")) \
    .withColumn("DAY", dayofmonth("SALES_DATE")) \
    .withColumn("WEEKDAY", dayofweek("SALES_DATE")) \
    .withColumnRenamed("SALES_DATE", "DATE")

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 9, Finished, Available, Finished)

In [10]:
# Join the fact data with dimension data
fact_sales_df = merged_data_df \
    .join(dim_product_df, merged_data_df["PRODUCT_NAME"] == dim_product_df["PRODUCT_NAME"], "inner") \
    .join(dim_customer_df, merged_data_df["CUSTOMER_NAME"] == dim_customer_df["CUSTOMER_NAME"], "inner") \
    .join(dim_date_df, merged_data_df["SALES_DATE"] == dim_date_df["DATE"], "inner") \
    .select(
        merged_data_df["SALES_ID"],
        merged_data_df["SALES_DATE"],
        dim_product_df["PRODUCT_NAME"],  # explicitly reference the column from dim_product_df
        dim_product_df["CATEGORY"],
        dim_product_df["SUB_CATEGORY"],
        dim_customer_df["CUSTOMER_NAME"],
        dim_customer_df["REGION"],
        dim_customer_df["CUSTOMER_TYPE"],
        merged_data_df["QUANTITY_SOLD"],
        merged_data_df["UNIT_PRICE"],
        merged_data_df["DISCOUNT"],
        merged_data_df["TOTAL_AMOUNT"],
        dim_date_df["YEAR"],
        dim_date_df["MONTH"],
        dim_date_df["DAY"],
        dim_date_df["WEEKDAY"]
    )

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 12, Finished, Available, Finished)

In [11]:
display(dim_product_df)

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e247973d-25e5-4ed5-8472-59628bb2f6d6)

In [12]:
display(dim_customer_df)

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 297159d8-6268-40a3-ad27-a640a4079179)

In [13]:
display(dim_date_df)

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ae84fb37-0a67-447a-9493-2b30b2d0c05e)

In [15]:
# Define the Gold layer path for the fact table
gold_layer_path = "Files/Gold/fact_sales"
fact_sales_df.write.mode("append").parquet(gold_layer_path)

# Define the Gold layer path for the Dim_Product dimension table
dim_product_gold_layer_path = "Files/Gold/dim_product"
dim_product_df.write.mode("append").parquet(dim_product_gold_layer_path)

# Define the Gold layer path for the Dim_Customer dimension table
dim_customer_gold_layer_path = "Files/Gold/dim_customer"
dim_customer_df.write.mode("append").parquet(dim_customer_gold_layer_path)

# Define the Gold layer path for the Dim_Date dimension table
dim_date_gold_layer_path = "Files/Gold/dim_date"
dim_date_df.write.mode("append").parquet(dim_date_gold_layer_path)

StatementMeta(, 3095b0c7-a737-48bf-a32c-ee74964a6bcd, 17, Finished, Available, Finished)